# 01 Multidimensional multi Knapsack Problem

## Problem Overview

The multidimensional multi knapsack problem (MMKP) is a generalization of the classic knapsack problem. In this problem, we have multiple knapsacks, each with its own capacity constraints across multiple dimensions (e.g., weight, volume, etc.). We also have a set of items, each with a value and a set of weights corresponding to each dimension. The goal is to maximize the total value of the items placed in the knapsacks without exceeding the capacity constraints of any knapsack in any dimension. 

### Attributes
- The `CONSTRAINTS` array define for each knapsack, what is the maximum weight and what volume it can carry.
- The `WEIGHTS` array defines for each item, what are the dimensions of the item (weight, volume, ...).
- The `VALUES` array defines for each item, what is its value.

### Example Output

with 3 kapsacks and 10 items with 2 dimensions (weight and volume):

```bash
(array([[287, 117],
        [319, 114],
        [204, 244]]),
 array([[52, 69],
        [13, 86],
        [89, 76],
        [54, 52],
        [25, 89],
        [ 5, 87],
        [50, 19],
        [97, 60],
        [85, 27],
        [97, 79]]),
 array([64, 73, 52, 71,  0, 95,  0, 34, 31, 21]))
```

> Note: It is need less to say that the items are the indecises of the arrays like 0, 1, 2, ..., n-1 in `VALUES` and `WEIGHTS`. **and one items can be in only one knapsack.**

In [6]:
import numpy as np
from tqdm import tqdm
from collections import deque

In [7]:
# Problem 1 
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 10
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)
# inital solution with empty knapsacks
solution = np.array(
    [[False for _ in range(NUM_ITEMS)] for _ in range(NUM_KNAPSACKS)] , dtype=np.bool
)

In [8]:
def value(solution : np.ndarray) -> int:
    return (VALUES * np.any(solution, axis=0)).sum()

def valid(solution : np.ndarray) -> bool:
    return np.all(WEIGHTS.T@solution.T <= CONSTRAINTS.T)

def get_items_left(solution: np.ndarray):
    '''
    Since each item can only be in one knapsack, we just check indices of current solution for all knapsacks are empty, if so, that item is left outside of knapsacks.
    '''
    items_in_knapsacks = np.any(solution, axis=0)
    return np.where(~items_in_knapsacks)[0]

In [22]:
def tweak(solution : np.ndarray) -> np.ndarray:
    '''
    tweak the solution by randomly assigning one **unassigned item** to a random knapsack.

    By checking unassigned items, we ensure that we do not put an item in two knapsacks.
    '''
    new_solution = solution.copy()
    # what are the items left to be assigned
    items_left = get_items_left(new_solution)
    if len(items_left) == 0:
        return new_solution
    # randomly pick one items in range [0, NUM_ITEMS)
    item = rng.choice(items_left) # this is the index in KANAPSACK
    # randomly pick one knapsack in range [0, NUM_KNAPSACKS)
    knapsack = rng.integers(0, NUM_KNAPSACKS)
    # assign the item to the knapsack
    new_solution[knapsack, item] = True
    
    return new_solution

def sa_tweak(solution : np.ndarray, strength : float = 0.5) -> np.ndarray:
    '''
    Simulated Annealing tweak
    '''
    
    new_solution = solution.copy()
    # what are the items left to be assigned
    items_left = get_items_left(new_solution)
    if len(items_left) == 0:
        return new_solution
    
    again = True
    while again:

        item = rng.choice(items_left) # this is the index in KANAPSACK
        knapsack = rng.integers(0, NUM_KNAPSACKS)
        new_solution[knapsack, item] = True
        again = rng.random() < strength

    return new_solution

In [14]:
def test_tweak(solution):
    new_solution = solution.copy()
    for _ in range(5):
        new_solution = tweak(new_solution)
        items_left = get_items_left(new_solution)

        print("Items left:", items_left, "Count:", len(items_left))
        print("Value:", value(new_solution))
        print("Valid:", valid(new_solution))
        # print(new_solution)
test_tweak(solution)

Items left: [0 1 2 4 5 6 7 8 9] Count: 9
Value: 43
Valid: True
Items left: [0 1 2 4 5 7 8 9] Count: 8
Value: 51
Valid: False
Items left: [0 1 2 4 5 7 8] Count: 7
Value: 60
Valid: False
Items left: [0 2 4 5 7 8] Count: 6
Value: 137
Valid: False
Items left: [0 2 4 7 8] Count: 5
Value: 222
Valid: False


In [23]:
from collections import deque

def hill_climbing(solution: np.ndarray, max_iter=1000, max_messages=5, print_every=100):
    best_solution = solution.copy()
    best_value = value(best_solution)
    messages = deque(maxlen=max_messages)
    
    for iteration in tqdm(range(max_iter), desc="Hill Climbing"):
        new_solution = tweak(best_solution)

        items_left = len(get_items_left(new_solution))
        
        if valid(new_solution):
            new_value = value(new_solution)
            if new_value > best_value:
                best_value = new_value
                best_solution = new_solution
                messages.append(f"Iter {iteration}: value = {best_value}, item left = {items_left}")
            if new_value == best_value:
                best_solution = new_solution

        if iteration % print_every == 0 and messages:
            tqdm.write("\n".join(messages))
            messages.clear()

        if items_left == 0:
            tqdm.write("All items assigned!")
            break

    # tqdm.write(f"Iter {iteration + 1}: Best value = {best_value}, items left = {len(get_items_left(best_solution))}")
    return best_solution, best_value, items_left

# Probem 1 Configuration

In [24]:
# Problem 1 
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)
# inital solution with empty knapsacks
solution = np.array(
    [[False for _ in range(NUM_ITEMS)] for _ in range(NUM_KNAPSACKS)] , dtype=np.bool
)

best_solution, best_value, items_left_num = hill_climbing(solution, max_iter=10000)
print(100 * "-")
print("Best value found for problem 1:", best_value)
if items_left_num != 0:
    print("But there are still", items_left_num, "items left unassigned.")

Hill Climbing:   0%|          | 26/10000 [00:00<00:03, 2507.29it/s]

Iter 0: value = 83, item left = 19
All items assigned!
----------------------------------------------------------------------------------------------------
Best value found for problem 1: 1022


**The Simulation Annealing hill climbing algorithm below is defined for next two problems:**

In [25]:
def sa_hill_climbing(solution: np.ndarray, max_iter=1000, max_messages=5,
                    print_every=100, initial_temp=1.0, cooling_rate=0.99):
    best_solution = solution.copy()
    best_value = value(best_solution)
    temp = initial_temp
    messages = deque(maxlen=max_messages)

    for iteration in tqdm(range(max_iter), desc="Simulated Annealing Hill Climbing"):
        new_solution = sa_tweak(best_solution, strength=0.8)
        items_left = len(get_items_left(new_solution))

        if valid(new_solution):
            new_value = value(new_solution)
            if new_value > best_value:
                best_value = new_value
                best_solution = new_solution
                messages.append(f"Iter {iteration}: value = {best_value}, item left = {items_left}")
            elif new_value == best_value: # moving is better than staying ! 
                best_solution = new_solution
            else: # SA 
                # accept with a probability of exp((new_value - best_value) / temp)
                prob = np.exp((new_value - best_value) / temp)
                if rng.random() < prob:
                    best_solution = new_solution

        temp *= cooling_rate

        if iteration % print_every == 0 and messages:
            tqdm.write("\n".join(messages))
            messages.clear()

        if items_left == 0:
            tqdm.write("All items assigned!")
            break

    return best_solution, best_value, items_left

# Problem 2 Configuration

In [ ]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)
# inital solution with empty knapsacks
solution = np.array(
    [[False for _ in range(NUM_ITEMS)] for _ in range(NUM_KNAPSACKS)] , dtype=np.bool
)

best_solution, best_value, items_left_num = sa_hill_climbing(solution, max_iter=100000,
                                                             initial_temp=10.0, cooling_rate=0.99)
print(100 * "-")
print("Best value found for problem 2:", best_value)
if items_left_num != 0:
    print("But there are still", items_left_num, "items left unassigned.")

Simulated Annealing Hill Climbing:   2%|▏         | 1614/100000 [00:00<00:12, 8095.66it/s]

Iter 0: value = 3554, item left = 93
Iter 57: value = 20004, item left = 60
Iter 61: value = 20442, item left = 59
Iter 68: value = 21237, item left = 58
Iter 75: value = 21937, item left = 57
Iter 83: value = 22776, item left = 56
Iter 115: value = 23835, item left = 54
Iter 150: value = 24310, item left = 53
Iter 234: value = 24712, item left = 52
Iter 257: value = 24806, item left = 51
Iter 273: value = 25704, item left = 50
Iter 277: value = 26067, item left = 49
Iter 352: value = 26908, item left = 48
Iter 372: value = 27097, item left = 47
Iter 530: value = 27566, item left = 46
Iter 626: value = 27793, item left = 45
Iter 660: value = 28243, item left = 44
Iter 965: value = 29183, item left = 43
Iter 1035: value = 29729, item left = 42
Iter 1432: value = 29814, item left = 41


Simulated Annealing Hill Climbing:  17%|█▋        | 16799/100000 [00:01<00:07, 10655.80it/s]

Iter 15145: value = 30168, item left = 40


Simulated Annealing Hill Climbing: 100%|██████████| 100000/100000 [00:09<00:00, 10675.66it/s]

----------------------------------------------------------------------------------------------------
Best value found for problem 2: 30168
But there are still 27 items left unassigned.


# Problem 3 

In [33]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)
# inital solution with empty knapsacks
solution = np.array(
    [[False for _ in range(NUM_ITEMS)] for _ in range(NUM_KNAPSACKS)] , dtype=np.bool
)

best_solution, best_value, items_left_num = sa_hill_climbing(solution, max_iter=1000,
                                                             initial_temp=10.0, cooling_rate=0.8)
print(100 * "-")
print("Best value found for problem 3:", best_value)
if items_left_num != 0:
    print("But there are still", items_left_num, "items left unassigned.")

Simulated Annealing Hill Climbing:   0%|          | 1/1000 [00:00<03:34,  4.65it/s]

Iter 0: value = 1709, item left = 4996


Simulated Annealing Hill Climbing:  10%|█         | 101/1000 [00:20<03:07,  4.79it/s]

Iter 96: value = 218049, item left = 4540
Iter 97: value = 221350, item left = 4535
Iter 98: value = 222695, item left = 4533
Iter 99: value = 223520, item left = 4530
Iter 100: value = 224024, item left = 4529


Simulated Annealing Hill Climbing:  20%|██        | 201/1000 [00:41<02:56,  4.52it/s]

...

Iter 780: value = 908960, item left = 3171
Iter 781: value = 912635, item left = 3165
Iter 787: value = 913133, item left = 3163
Iter 793: value = 914419, item left = 3161
Iter 795: value = 915281, item left = 3159


Simulated Annealing Hill Climbing:  90%|█████████ | 901/1000 [03:08<00:22,  4.47it/s]

Iter 879: value = 930964, item left = 3133
Iter 883: value = 931126, item left = 3132
Iter 889: value = 931567, item left = 3131
Iter 894: value = 932186, item left = 3130
Iter 897: value = 932276, item left = 3129


Simulated Annealing Hill Climbing: 100%|██████████| 1000/1000 [03:28<00:00,  4.79it/s]

----------------------------------------------------------------------------------------------------
Best value found for problem 3: 941067
But there are still 3108 items left unassigned.
